In [1]:
#Se importan las librerias para trabajar
import pandas as pd
import requests

In [2]:
#Se guarda la url para poder buscar las referencias geograficas
url = 'https://apis.datos.gob.ar/georef/api/departamentos'

In [3]:
#Para realizar el metodo post a departamentos se pueden espeficiar los parámetros. 
#Se crea la variable paramtros para pasar los parámetros e incremtar el numero de respuetas obtenidas.
parametros = {
    "departamentos": [
        {
            #"id": "string",
            #"nombre": "string",
            #"provincia": "Santa Fe",
            #"interseccion": "provincia:82,departamento:82084,municipio:820196",
            "orden": "id",
            "aplanar": True,
            "campos": "estandar",
            "max": 600,
            "inicio": 0,
            "exacto": True
        }
    ]
}

In [4]:
#Se realiza una petición para obtener la respuesta
try:
    response = requests.post(url, json=parametros)
    response.raise_for_status()
    print(f'Solicitud correcta. Statuscode: {response.status_code}')
except requests.exceptions.RequestException as e:
    print(f'Error en la solicitud: {e}')

Solicitud correcta. Statuscode: 200


In [5]:
#Se guarda la variable como json para poder trabajarla
responsejson = response.json()

In [6]:
#Se guardan los resultados de los departamentos en una variable.
infoDepartamentos = responsejson['resultados'][0]['departamentos']

In [7]:
infoDepartamentos[-3]

{'centroide_lat': -54.7704623704114,
 'centroide_lon': -66.6837162279656,
 'id': '94015',
 'nombre': 'Ushuaia',
 'provincia_id': '94',
 'provincia_nombre': 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'}

In [8]:
#Se crea un data frame con la información de los departamentos. Esto luego se va a utilizar para obtener las coordenadas
#y combinar esas coordenadas con los departamentos de los partidos.
dfDepartamentos = pd.DataFrame(infoDepartamentos)

In [9]:
#Se carga el archivo de estimaciones agricolas para poder crear el data frame y combinarlo para obtener las coodernadas
#Los datos fueron descargados desde https://geoportal.magyp.gob.ar/tematizador/agricola/
archivoEstimaciones = 'estimacionesAgricolas2023.xlsx'
dfEstimaciones = pd.read_excel(archivoEstimaciones)

In [10]:
dfDepartamentos.columns

Index(['centroide_lat', 'centroide_lon', 'id', 'nombre', 'provincia_id',
       'provincia_nombre'],
      dtype='object')

In [11]:
dfEstimaciones.columns

Index(['id_provincia', 'provincia', 'id_departamento', 'departamento',
       'id_cultivo', 'cultivo', 'id_campaña', 'campaña', 'sup_sembrada',
       'sup_cosechada', 'produccion ', 'rendimiento'],
      dtype='object')

In [12]:
#Los data frames tienen nombres de la columnas distintas por lo que hay que modificar los nombres de las columnas
#para poder realizar las comparaciones entre las mismas y trabajarlas. En este caso la comparación que queremos realizar es entre los id
#de "provincia" y "id_departamento" entonces tenemos que hacer que los nombres sean iguales
dfDepartamentos.rename(columns={'provincia_id':'id_provincia',
                               'id':'id_departamento'},
                      inplace=True)

In [13]:
dfDepartamentos.columns

Index(['centroide_lat', 'centroide_lon', 'id_departamento', 'nombre',
       'id_provincia', 'provincia_nombre'],
      dtype='object')

In [14]:
dfEstimaciones.columns

Index(['id_provincia', 'provincia', 'id_departamento', 'departamento',
       'id_cultivo', 'cultivo', 'id_campaña', 'campaña', 'sup_sembrada',
       'sup_cosechada', 'produccion ', 'rendimiento'],
      dtype='object')

In [15]:
#Es df de estimaciones tiene valores para id_departamento con el valor 0, que su nombre de departamento figura como "sin definir".
#Se realizó una busqueda y no hay departamento con id = 0 en las base de datos u otros archivos.
#Se decide seleccionar todas esas columnas y eliminarlas del df. Son 592 datos en total los que se descartan
dfEstimaciones.drop(dfEstimaciones[dfEstimaciones['id_departamento'] == 0].index, inplace=True)

In [16]:
#Se trabaja sobre el df de departamentos.
#El id departamento viene enlazado primero con el código de la provincia por lo que hay que descartarlo para convertirlo y que sea igual al que figura
#en el df de estimaciones
#Luego de descartar esos valores se convierten los valores a int64 para que sean iguales a los del df de estimaciones. 
#Esto se hace para luego poder comparar y realiar un merge de ambos data frames.
dfDepartamentos['id_departamento'] = dfDepartamentos['id_departamento'].str[-3:]
dfDepartamentos['id_departamento'] = dfDepartamentos['id_departamento'].astype('int64')
dfDepartamentos['id_provincia'] = dfDepartamentos['id_provincia'].astype('int64')

In [17]:
dfDepartamentos['id_departamento'].dtype

dtype('int64')

In [18]:
dfEstimaciones['id_departamento'].dtype

dtype('int64')

In [19]:
#Se hace una combinación de los dos df. Se busca que matcheen las columnas id_departamento e id_provincia.
#Si para esas columnas el match es True se agregan esas filas al nuevo df
dfEstimacionesConCoordenadas = pd.merge(dfEstimaciones, dfDepartamentos, on=['id_departamento', 'id_provincia'], how='left')

In [20]:
#El campo del texto campaña figura con la siguiente expresión 'año1/año2'.
#Nos interesa quedarnos unicamente con el primer año para luego poder trabajar
dfEstimacionesConCoordenadas['campaña'] = dfEstimacionesConCoordenadas['campaña'].str[:4]

In [21]:
#El dataframe obtenido tiene algunos valores que no están definido. En la columna rendimiento algunas filas no tienen su valor
#Por lo que se las excluye para que luego no generen problemas el realizar los modelados de datos
dfEstimacionesConCoordenadas = dfEstimacionesConCoordenadas[dfEstimacionesConCoordenadas['rendimiento'] != 'SD']

In [22]:
#Se obtiene el nombre de las columnas del dataframe. Para luego poder orderanlos según la columna.
#Se lo guarda en una variable para que sea más fácil de modificar ante cambios futuros.

nombreColumnas = dfEstimacionesConCoordenadas.columns

#Se imprimen los nombres de las columnas con su índices para luego poder acceder a la variable previamente definida

for index, nombre in enumerate(nombreColumnas):
    print(f'indice columna: {index}, nombre: {nombre}')

indice columna: 0, nombre: id_provincia
indice columna: 1, nombre: provincia
indice columna: 2, nombre: id_departamento
indice columna: 3, nombre: departamento
indice columna: 4, nombre: id_cultivo
indice columna: 5, nombre: cultivo
indice columna: 6, nombre: id_campaña
indice columna: 7, nombre: campaña
indice columna: 8, nombre: sup_sembrada
indice columna: 9, nombre: sup_cosechada
indice columna: 10, nombre: produccion 
indice columna: 11, nombre: rendimiento
indice columna: 12, nombre: centroide_lat
indice columna: 13, nombre: centroide_lon
indice columna: 14, nombre: nombre
indice columna: 15, nombre: provincia_nombre


In [23]:
#Se especifica el orden y las columnas como se quiere obtener el dataframe
ordenColumnas = [nombreColumnas[1],
                 nombreColumnas[0],
                 nombreColumnas[3],
                 nombreColumnas[2],
                 nombreColumnas[5],
                 nombreColumnas[8],
                 nombreColumnas[9],
                 nombreColumnas[10],
                 nombreColumnas[11],
                 nombreColumnas[7],
                 nombreColumnas[12],
                 nombreColumnas[13]
                ]

In [24]:
#Se aplica el orden de las columnas al dataframe. Tener en cuenta que las columnas que no se seleccionan no van a figurar en el dataframe
dfEstimacionesConCoordenadas = dfEstimacionesConCoordenadas[ordenColumnas]

In [25]:
#Se ordena el df por columnas
dfEstimacionesConCoordenadas = dfEstimacionesConCoordenadas.sort_values(['provincia','departamento', 'cultivo', 'campaña'])

In [26]:
#Se crea un diccionario con categorias para los cultivos, para luego agregar una columna de categorias de cultivos
cultivos = {
    'Ajo': 'Hortalizas',
    'Algodón': 'Industriales',
    'Alpiste': 'Cereales',
    'Arroz': 'Cereales',
    'Arveja': 'Legumbres',
    'Avena': 'Cereales',
    'Banana': 'Frutales',
    'Caña de azúcar': 'Industriales',
    'Cártamo': 'Industriales',
    'Cebada total': 'Cereales',
    'Cebada cervecera': 'Cereales',
    'Cebada forrajera': 'Cereales',
    'Cebolla total': 'Hortalizas',
    'Centeno': 'Cereales',
    'Colza': 'Oleaginosas',
    'Garbanzo': 'Legumbres',
    'Girasol': 'Oleaginosas',
    'Jojoba': 'Frutales',
    'Lenteja': 'Legumbres',
    'Limón': 'Frutales',
    'Lino': 'Oleaginosas',
    'Maíz': 'Cereales',
    'Mandarina': 'Frutales',
    'Maní': 'Oleaginosas',
    'Mijo': 'Cereales',
    'Naranja': 'Frutales',
    'Papa total': 'Hortalizas',
    'Pomelo': 'Frutales',
    'Poroto alubia': 'Legumbres',
    'Poroto negro': 'Legumbres',
    'Poroto otros': 'Legumbres',
    'Poroto seco': 'Legumbres',
    'Poroto total': 'Legumbres',
    'Soja total': 'Oleaginosas',
    'Soja 1ra': 'Oleaginosas',
    'Soja 2da':'Oleaginosas',
    'Sorgo': 'Cereales',
    'Té': 'Industriales',
    'Trigo candeal': 'Cereales',
    'Trigo total': 'Cereales',
    'Tung': 'Oleaginosas',
    'Yerba mate': 'Industriales'
}

In [27]:
#Se crea una categoría de cultivos al df final.
dfEstimacionesConCoordenadas['Categoria cultivo'] = dfEstimacionesConCoordenadas['cultivo'].map(cultivos)

In [28]:
dfEstimacionesConCoordenadas['Categoria cultivo'].unique()

array(['Hortalizas', 'Cereales', 'Legumbres', 'Oleaginosas',
       'Industriales', 'Frutales'], dtype=object)

In [29]:
#Se exporta el dataframe en un archivo Excel
dfEstimacionesConCoordenadas.to_excel("estimacionesConCoordenadas.xlsx", index=False)